In [ ]:
# Multiple Outputs
import numpy as np
import os
import cv2

## Dataset 

In [ ]:
FACES_PATH = '../data/face_detection/faces/'

### Read training dataset

In [ ]:
training_size = 1000

def read_pos_images():
    #Read positive images:
    path, __, filenames = next(os.walk(FACES_PATH+'pos_train/'))
    file_count = training_size #len(filenames)
    images = np.empty([0,12,3])
    for i in range(file_count):
        j=i+1
        img=cv2.imread(f"{path}{j}.bmp")
        images=np.append(images,img,axis=0)
    #Create list of probabilities:
    prob=[]
    for i in range(file_count):
        prob.append([[[0.0,1.0]]])
    #Create list of coordinates:
    coordinates=[]
    file = open(FACES_PATH+'coordinates.txt','r')
    lines = file.readlines()
    lines = [line[:-1] for line in lines]
    #idx=[1,0,3,2]
    idx=[0,1,2,3]
    f_count = 0
    for line in lines:
        line = line.split(" ")
        line = line[1]
        line=line[1:-1]
        line = line.split(",")
        #Transpose coordinates
        x=0
        nline=[]
        for i in idx:
            nline.append(line[i])
            x=x+1
        line=[[[float(c) for c in nline]]]
        coordinates.append(line)
        f_count = f_count+1
        if f_count == file_count:
            break
    #Return images, probs, and coordinates
    return images, prob, coordinates

def read_neg_images():
    #Read negative images:
    path, __, filenames = next(os.walk(FACES_PATH+'neg_train/'))
    file_count = training_size #len(filenames)
    images = np.empty([0,12,3])
    for i in range(file_count):
        j=i+1
        img=cv2.imread(f"{path}{j}.bmp")
        images=np.append(images,img,axis=0)
    #Create list of probabilities:
    prob=[]
    for i in range(file_count):
        prob.append([[[1.0,0.0]]])
    #Create list of coordinates:
    coordinates=[]
    for i in range(file_count):
        coordinates.append([[[0.0,0.0,0.0,0.0]]])
    #Return images, prob, coordinates
    return images, prob, coordinates

#Read in all images, probabilities, and coordinates
pimages, pprob, pcoordinates = read_pos_images()
nimages, nprob, ncoordinates = read_neg_images()
o_images=np.append(pimages,nimages,axis=0)
o_images=np.reshape(o_images,(-1,12,12,3))
o_prob=pprob+nprob
o_coordinates=pcoordinates+ncoordinates

#Shuffle them up using an index
idx=np.arange(len(o_prob))
np.random.shuffle(idx)
images=np.empty_like(o_images)
c=0
for i in idx:
    images[c]=o_images[i]
    c=c+1
#images=(np.float32)(images-127.5)/128.0
images=(np.float32)(images)/255

#images = np.transpose(images, (0, 2, 1, 3)) #Transpose images
prob=[]
for i in idx:
    prob.append(o_prob[i])
coordinates=[]
for i in idx:
    coordinates.append(o_coordinates[i])

In [ ]:
print('X_train , Image batch shape ', images.shape)
print('y_train , Classification ground true batch shape ' ,np.array(prob).shape)
print('y_train , Coordinates ground true batch shape ', np.array(coordinates).shape)

## Create X_data for train and validation

In [ ]:
X_data = images

In [ ]:
print('X_data shape',X_data.shape)

In [ ]:
del images

## Create "y_data" for train and validation

In [ ]:
y_data = np.concatenate((np.array(prob), np.array(coordinates)), axis=3)

In [ ]:
print('y_data shape',y_data.shape)

In [ ]:
print('y_data Classification shape', y_data[:,:,:,:2].shape)
print('y_data Coordinate shape',y_data[:,:,:,2:].shape)

## Divide dataset to "train', "val" and "test"

In [ ]:
def load_data(X, y, training_prec = 0.7, val_prec = 0.1, test_prec = 0.2):
        data_length = len(X)
        num_training = np.int(data_length * training_prec)
        num_validation = np.int(data_length * val_prec)
        
        mask = range(num_training)
        X_train = X[mask]
        y_train = y[mask]
        mask = range(num_training, num_training + num_validation)
        X_val = X[mask]
        y_val = y[mask]
        mask = range(num_training + num_validation, data_length)
        X_test = X[mask]
        y_test = y[mask]
        
        return X_train, y_train, X_val, y_val, X_test, y_test


In [ ]:
X_train, y_train, X_val, y_val, X_test, y_test = load_data(X_data, y_data)
print('Train data shape: ', X_train.shape)
print('Train labels shape: ', y_train.shape, y_train.dtype)
print('Validation data shape: ', X_val.shape)
print('Validation labels shape: ', y_val.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)

## Build P-Net Keras model

In [ ]:
import tensorflow as tf
from keras.utils import plot_model
from keras.models import Model
from keras.layers import MaxPooling2D, Conv2D, Input, Layer, Concatenate, concatenate
from keras.layers.advanced_activations import PReLU
#from keras.layers.wrappers import TimeDistributed

In [ ]:
def PNet():
    
    initializer = tf.keras.initializers.VarianceScaling(scale=2.0)

    #compute the loss function over the classification and over bounding box 
    classification_loss = tf.keras.losses.BinaryCrossentropy()
    bbox_loss = tf.keras.losses.MeanSquaredError()
    

    #input layer
    visible = Input(shape=(12,12,3))
    
    # CNN Stage 1
    conv1 = Conv2D(10, kernel_size=(3,3))(visible)
    prelu1 = PReLU(alpha_initializer='zero', alpha_regularizer=None, alpha_constraint=None, shared_axes=[1,2])(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(prelu1)
   
    #CNN Stage 2
    conv2 = Conv2D(16, kernel_size=(3,3))(pool1)
    prelu2 = PReLU(alpha_initializer='zero', alpha_regularizer=None, alpha_constraint=None, shared_axes=[1,2])(conv2)
    
    # CNN stage 3
    conv3 = Conv2D(32, kernel_size=(3,3),)(prelu2)
    prelu3 = PReLU(alpha_initializer='zero', alpha_regularizer=None, alpha_constraint=None, shared_axes=[1,2])(conv3)
    
    pred_classification = Conv2D(2, kernel_size=(1,1), activation='softmax', name='classification')(prelu3)
    pred_bbox = Conv2D(4, kernel_size=(1,1), name='bbox')(prelu3)
    
    model = Model(inputs=visible, outputs=[pred_classification, pred_bbox])
                  
 
    # Define P-Net Loss : BinaryCrossEntropy(Classification) + 0.5 * MSE(bounding_bbox) 
    def PNet_loss():
            #Create a loss function for P-Net
        def loss(y_true,y_pred):
            prediction_loss = classification_loss(y_classification, pred_classification)
            coordinate_loss = bbox_loss(pred_bbox, y_bbox)
            return prediction_loss + 0.5 * coordinate_loss * y_classification[:,:,:,1]
   
        # Return a function
        return loss
            
 
    # create placeholder for targets
    y_classification = tf.keras.backend.placeholder(dtype='float32', shape=pred_classification.shape) # shapes of output1 your target has
    y_bbox = tf.keras.backend.placeholder(dtype='float32', shape=pred_bbox.shape) # shapes of output2 your target has
    
    
    learning_rate = 1e-3
    adam = tf.keras.optimizers.Adam(learning_rate)
    
    model.compile(optimizer=adam, 
                  loss = PNet_loss(),
                  target_tensors=[y_classification,y_bbox],
                  metrics=[['accuracy'],['mse']])
    # summarize layers
    print(model.summary())
    # plot graph
    plot_model(model, to_file='multiple_outputs.png')
    
    return model

In [ ]:
model = PNet()

In [ ]:
model.fit(X_train, [y_train[:,:,:,:2],y_train[:,:,:,2:]], batch_size=64, epochs=50,
          validation_data=(X_val, [y_val[:,:,:,:2],y_val[:,:,:,2:]]))

### Test the PNet  to ensure that the implementation does not crash and produces outputs of the expected shape.
Pnet will output are:
1. Face classification,  size (batch,1,1,2) for 2 calss classification, "Face", and "Not face"
2. Bounding box  (batch,1,1,4) for 4 boundind box corrdinates (x,y,w,h)

In [ ]:
model.save('P-Net.h5')

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd


In [ ]:
metrics = pd.DataFrame(model.history.history)

In [ ]:
metrics.head()

In [ ]:
metrics[['loss', 'val_loss']].plot(figsize=(12,8))

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
predictions = model.predict(X_test)

In [ ]:
predictions.shape

In [ ]:
predictions = np.squeeze(predictions)

In [ ]:
y_test = np.squeeze(y_test)

In [ ]:
print(predictions.shape, y_test.shape)

In [ ]:
#predictions[:10,1:2]

In [ ]:
#y_test[:10,1:2]

In [ ]:
#np.round(predictions[:10,1:2])

In [ ]:
print(classification_report(y_test[:,:2], np.round(predictions[:,:2])))

In [ ]:
# Not face
print(confusion_matrix(y_test[:,0:1], np.round(predictions[:,0:1])))

In [ ]:
#  face
print(confusion_matrix(y_test[:,1:2], np.round(predictions[:,1:2])))

In [ ]:
test_face = X_test[0]

In [ ]:
plt.figure(figsize=(2,2))
plt.imshow(test_face)